In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

# Define the file paths
file_path1 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_training_data_for_mode.csv'
file_path2 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_testing_data_for_mode.csv'
file_path3 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_training_data_for_mode.csv'
file_path4  = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_testing_data_for_mode.csv'
# Use pandas to read the CSV files and then convert them to NumPy arrays
Caltech_data_training = pd.read_csv(file_path1).values
Caltech_data_testing = pd.read_csv(file_path2).values

JPL_training_data = pd.read_csv(file_path3).values
JPL_testing_data=pd.read_csv(file_path4).values

In [ ]:
def convert_to_interval(datetime_str):
    # Extract time string from the datetime string
    time_str = datetime_str.split()[1]

    # Convert time string into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(':'))

    # Calculate total minutes
    total_minutes = hours * 60 + minutes

    # Find the nearest lower multiple of 15 for the minutes
    interval_minutes = (total_minutes // 15) * 15

    # Convert back to hour and minute
    interval_hour, interval_minute = divmod(interval_minutes, 60)

    # Return as a formatted string
    return "{:02d}:{:02d}".format(interval_hour, interval_minute)

# Apply the conversion to the data
Caltech_data_training[:, 1] = np.vectorize(convert_to_interval)(Caltech_data_training[:, 1])
Caltech_data_testing[:, 1] = np.vectorize(convert_to_interval)(Caltech_data_testing[:, 1])

JPL_training_data[:, 1] = np.vectorize(convert_to_interval)(JPL_training_data[:, 1])
JPL_testing_data[:, 1] = np.vectorize(convert_to_interval)(JPL_testing_data[:, 1])

In [ ]:
Caltech_data_training=Caltech_data_training[:,1:]
Caltech_data_testing=Caltech_data_testing[:,1:]

JPL_training_data=JPL_training_data[:,1:]
JPL_testing_data=JPL_testing_data[:,1:]

In [ ]:
Caltech_data_training=np.delete(Caltech_data_training, [1,-1], axis=1)
Caltech_data_testing=np.delete(Caltech_data_testing, [1,-1], axis=1)

JPL_training_data=np.delete(JPL_training_data, [1,-1], axis=1)
JPL_testing_data=np.delete(JPL_testing_data, [1,-1], axis=1)

In [ ]:
Caltech_data_training[:, -1] = Caltech_data_training[:, -1].astype(float) * 60 #convert duration in minutes
Caltech_data_testing[:, -1] = Caltech_data_testing[:, -1].astype(float) * 60

JPL_training_data[:, -1] = JPL_training_data[:, -1].astype(float) * 60
JPL_testing_data[:, -1] = JPL_testing_data[:, -1].astype(float) * 60

In [ ]:
# Convert time strings to total minutes for sorting
total_minutes_caltech_training = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in Caltech_data_training[:, 0]])
total_minutes_caltech_testing = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in Caltech_data_testing[:, 0]])

total_minutes_JPL_training = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in JPL_training_data[:, 0]])
total_minutes_JPL_testing = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in JPL_testing_data[:, 0]])

# Argsort first by the 3rd column and then by total_minutes
indices_caltech_training = np.lexsort((total_minutes_caltech_training, Caltech_data_training[:, 2].astype(int)))
indices_caltech_testing = np.lexsort((total_minutes_caltech_testing, Caltech_data_testing[:, 2].astype(int)))

indices_JPL_training = np.lexsort((total_minutes_JPL_training, JPL_training_data[:, 2].astype(int)))
indices_JPL_testing = np.lexsort((total_minutes_JPL_testing, JPL_testing_data[:, 2].astype(int)))
# Use the sorted indices to reorder the array
sorted_data_caltech_training = Caltech_data_training[indices_caltech_training]
sorted_data_caltech_testing = Caltech_data_testing[indices_caltech_testing]

sorted_data_JPL_training = JPL_training_data[indices_JPL_training]
sorted_data_JPL_testing = JPL_testing_data[indices_JPL_testing]

In [ ]:
sorted_data_caltech_training[:, 3] = np.round(sorted_data_caltech_training[:, 3].astype(float) / 10) * 10
sorted_data_caltech_testing[:, 3] = np.round(sorted_data_caltech_testing[:, 3].astype(float) / 10) * 10

sorted_data_JPL_training[:, 3] = np.round(sorted_data_JPL_training[:, 3].astype(float) / 10) * 10
sorted_data_JPL_testing[:, 3] = np.round(sorted_data_JPL_testing[:, 3].astype(float) / 10) * 10

In [ ]:
#Round energy delivered to the nearest 0.5
for row in sorted_data_caltech_training:
    row[1] = round(row[1] * 2) / 2
for row in sorted_data_caltech_testing:
    row[1] = round(row[1] * 2) / 2

for row in sorted_data_JPL_training:
    row[1] = round(row[1] * 2) / 2
for row in sorted_data_JPL_testing:
    row[1] = round(row[1] * 2) / 2

In [ ]:
sorted_data_caltech_training

In [ ]:
import numpy as np
from scipy.stats import mode

unique_times = np.unique(sorted_data_caltech_training[:, 0])
results = []

for time in unique_times:
    subset = sorted_data_caltech_training[sorted_data_caltech_training[:, 0] == time]
    energy = subset[:, 1].astype(float)

    # Find the mode
    (values, counts) = np.unique(energy, return_counts=True)
    max_count = np.max(counts)
    modes = values[counts == max_count]

    # Average if there are multiple modes
    avg_mode = np.mean(modes)

    results.append([time, avg_mode])

population_mode_caltech = np.array(results)


print(population_mode_caltech)

In [ ]:
#Mode for Caltech (repeat)
import numpy as np

# Generating all possible 15-minute intervals in a 24-hour day
all_intervals = [f"{str(h).zfill(2)}:{str(m).zfill(2)}" for h in range(24) for m in [0, 15, 30, 45]]

def compute_mode(data):
    (values, counts) = np.unique(data, return_counts=True)
    max_count = np.max(counts)
    modes = values[counts == max_count]
    return np.mean(modes) if len(modes) > 0 else None

# Compute mode for each unique interval present in the data
modes_dict = {}
unique_times_in_data = np.unique(sorted_data_caltech_training[:, 0])
for time in unique_times_in_data:
    subset = sorted_data_caltech_training[sorted_data_caltech_training[:, 0] == time]
    energy = subset[:, 1].astype(float)
    current_mode = compute_mode(energy)

    if current_mode is not None:
        modes_dict[time] = current_mode

# For each interval in the 24-hour day, find mode or the mode of the closest preceding interval
results = []
last_known_mode = None
for interval in all_intervals:
    if interval in modes_dict:
        results.append([interval, modes_dict[interval]])
        last_known_mode = modes_dict[interval]
    else:
        results.append([interval, last_known_mode])

population_mode_caltech_repeat = np.array(results)
print(population_mode_caltech_repeat)

In [ ]:
#Mode for JPL (repeat)
import numpy as np

# Generating all possible 15-minute intervals in a 24-hour day
all_intervals = [f"{str(h).zfill(2)}:{str(m).zfill(2)}" for h in range(24) for m in [0, 15, 30, 45]]

def compute_mode(data):
    (values, counts) = np.unique(data, return_counts=True)
    max_count = np.max(counts)
    modes = values[counts == max_count]
    return np.mean(modes) if len(modes) > 0 else None

# Compute mode for each unique interval present in the data
modes_dict = {}
unique_times_in_data = np.unique(sorted_data_JPL_training[:, 0])
for time in unique_times_in_data:
    subset = sorted_data_JPL_training[sorted_data_JPL_training[:, 0] == time]
    energy = subset[:, 1].astype(float)
    current_mode = compute_mode(energy)

    if current_mode is not None:
        modes_dict[time] = current_mode

# For each interval in the 24-hour day, find mode or the mode of the closest preceding interval
results = []
last_known_mode = None
for interval in all_intervals:
    if interval in modes_dict:
        results.append([interval, modes_dict[interval]])
        last_known_mode = modes_dict[interval]
    else:
        results.append([interval, last_known_mode])

population_mode_JPL_repeat = np.array(results)
print(population_mode_JPL_repeat)

In [ ]:
# Mode for caltech (interpolation)
import numpy as np

# Generating all possible 15-minute intervals in a 24-hour day
all_intervals = [f"{str(h).zfill(2)}:{str(m).zfill(2)}" for h in range(24) for m in [0, 15, 30, 45]]

def compute_mode(data):
    (values, counts) = np.unique(data, return_counts=True)
    max_count = np.max(counts)
    modes = values[counts == max_count]
    return np.mean(modes) if len(modes) > 0 else None

# Compute mode for each unique interval present in the data
modes_dict = {}
unique_times_in_data = np.unique(sorted_data_caltech_training[:, 0])
for time in unique_times_in_data:
    subset = sorted_data_caltech_training[sorted_data_caltech_training[:, 0] == time]
    energy = subset[:, 1].astype(float)
    current_mode = compute_mode(energy)

    if current_mode is not None:
        modes_dict[time] = current_mode

# Linear interpolation function
def interpolate(start_time, end_time, start_val, end_val, current_time):
    start_hour, start_min = map(int, start_time.split(':'))
    end_hour, end_min = map(int, end_time.split(':'))
    current_hour, current_min = map(int, current_time.split(':'))

    # Convert times to minutes since midnight for easier calculations
    start_minutes = start_hour * 60 + start_min
    end_minutes = end_hour * 60 + end_min
    current_minutes = current_hour * 60 + current_min

    slope = (end_val - start_val) / (end_minutes - start_minutes)
    interpolated_val = start_val + slope * (current_minutes - start_minutes)

    return interpolated_val

results = []
last_known_time = None
last_known_mode = None
for interval in all_intervals:
    if interval in modes_dict:
        results.append([interval, modes_dict[interval]])
        last_known_time = interval
        last_known_mode = modes_dict[interval]
    else:
        # Find the next known interval after the current interval
        next_known_time = next((time for time in all_intervals if time > interval and time in modes_dict), None)
        if last_known_time and next_known_time:
            interpolated_value = interpolate(last_known_time, next_known_time, last_known_mode, modes_dict[next_known_time], interval)
            results.append([interval, interpolated_value])
        else:
            # If there's no previous or next known point, use the most recent known mode
            results.append([interval, last_known_mode])

population_mode_caltech_interpolation = np.array(results)
print(population_mode_caltech_interpolation)


In [ ]:
# Mode for JPL (interpolation)
import numpy as np

# Generating all possible 15-minute intervals in a 24-hour day
all_intervals = [f"{str(h).zfill(2)}:{str(m).zfill(2)}" for h in range(24) for m in [0, 15, 30, 45]]

def compute_mode(data):
    (values, counts) = np.unique(data, return_counts=True)
    max_count = np.max(counts)
    modes = values[counts == max_count]
    return np.mean(modes) if len(modes) > 0 else None

# Compute mode for each unique interval present in the data
modes_dict = {}
unique_times_in_data = np.unique(sorted_data_JPL_training[:, 0])
for time in unique_times_in_data:
    subset = sorted_data_JPL_training[sorted_data_JPL_training[:, 0] == time]
    energy = subset[:, 1].astype(float)
    current_mode = compute_mode(energy)

    if current_mode is not None:
        modes_dict[time] = current_mode

# Linear interpolation function
def interpolate(start_time, end_time, start_val, end_val, current_time):
    start_hour, start_min = map(int, start_time.split(':'))
    end_hour, end_min = map(int, end_time.split(':'))
    current_hour, current_min = map(int, current_time.split(':'))

    # Convert times to minutes since midnight for easier calculations
    start_minutes = start_hour * 60 + start_min
    end_minutes = end_hour * 60 + end_min
    current_minutes = current_hour * 60 + current_min

    slope = (end_val - start_val) / (end_minutes - start_minutes)
    interpolated_val = start_val + slope * (current_minutes - start_minutes)

    return interpolated_val

results = []
last_known_time = None
last_known_mode = None
for interval in all_intervals:
    if interval in modes_dict:
        results.append([interval, modes_dict[interval]])
        last_known_time = interval
        last_known_mode = modes_dict[interval]
    else:
        # Find the next known interval after the current interval
        next_known_time = next((time for time in all_intervals if time > interval and time in modes_dict), None)
        if last_known_time and next_known_time:
            interpolated_value = interpolate(last_known_time, next_known_time, last_known_mode, modes_dict[next_known_time], interval)
            results.append([interval, interpolated_value])
        else:
            # If there's no previous or next known point, use the most recent known mode
            results.append([interval, last_known_mode])

population_mode_JPL_interpolation = np.array(results)
print(population_mode_JPL_interpolation)


In [ ]:
import numpy as np

def smape(predicted, actual):
    return (np.abs(predicted - actual)) / (np.abs(predicted + actual))

# Map of predicted energies
predicted_map_caltech_repeat = {row[0]: float(row[1]) for row in population_mode_caltech_repeat}
predicted_map_JPL_repeat = {row[0]: float(row[1]) for row in population_mode_JPL_repeat}

predicted_map_caltech_interpolate = {row[0]: float(row[1]) for row in population_mode_caltech_interpolation}
predicted_map_JPL_interpolate= {row[0]: float(row[1]) for row in population_mode_JPL_interpolation}

# List of SMAPE values for each row
smape_values_caltech_repeat = [smape(predicted_map_caltech_repeat[row[0]], float(row[1])) for row in sorted_data_caltech_testing]
smape_values_JPL_repeat = [smape(predicted_map_JPL_repeat[row[0]], float(row[1])) for row in sorted_data_JPL_testing]

smape_values_caltech_interpolate = [smape(predicted_map_caltech_interpolate[row[0]], float(row[1])) for row in sorted_data_caltech_testing]
smape_values_JPL_interpolate = [smape(predicted_map_JPL_interpolate[row[0]], float(row[1])) for row in sorted_data_JPL_testing]

# Average SMAPE across all users and sessions
smape_caltech_repeat= np.mean(smape_values_caltech_repeat) * 100  # Multiply by 100 to express it in percentage
smape_JPL_repeat= np.mean(smape_values_JPL_repeat) * 100  # Multiply by 100 to express it in percentage

smape_caltech_interpolate= np.mean(smape_values_caltech_interpolate) * 100  # Multiply by 100 to express it in percentage
smape_JPL_interpolate= np.mean(smape_values_JPL_interpolate) * 100  # Multiply by 100 to express it in percentage

print(f"Population SMAPE Caltech (repeat): {smape_caltech_repeat:.2f}%")
print(f"Population SMAPE JPL (repeat): {smape_JPL_repeat:.2f}%")
print(f"Population SMAPE Caltech (interpolate): {smape_caltech_interpolate:.2f}%")
print(f"Population SMAPE JPL (interpolate): {smape_JPL_interpolate:.2f}%")